<a href="https://colab.research.google.com/github/vladgap/LLC/blob/main/Rotem%20LLC%20Extraction%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn import preprocessing
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"

In [9]:
!wget -O MLNN.py "https://raw.githubusercontent.com/vladgap/MLNN/main/MLNN_3_2.py"
from MLNN import *

--2025-02-10 11:37:31--  https://raw.githubusercontent.com/vladgap/MLNN/main/MLNN_3_2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14169 (14K) [text/plain]
Saving to: ‘MLNN.py’

MLNN.py             100%[===================>]  13.84K  --.-KB/s    in 0s      

2025-02-10 11:37:31 (27.3 MB/s) - ‘MLNN.py’ saved [14169/14169]



In [10]:
!wget -O LLC.py "https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_5_1.py"
import LLC

--2025-02-10 11:37:31--  https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_5_1.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22228 (22K) [text/plain]
Saving to: ‘LLC.py’

LLC.py              100%[===================>]  21.71K  --.-KB/s    in 0.002s  

2025-02-10 11:37:32 (12.3 MB/s) - ‘LLC.py’ saved [22228/22228]



In [11]:
# import LLC_5_2 as LLC52

In [12]:
def CopyPasteToArray(a):
  if a.startswith('\n'):
    b=a[1:]
  else:
    b=a
  if b.endswith('\n'):
    b=b[:-1]
  c=b.replace('\t',',')
  d=c.split('\n')
  f=[]
  for e in d:
    if e.replace(',','').replace('.','').isdigit(): # only digits no letters
      f.append(list(eval(e)))
    else:
      f.append(e.split(','))
  return f

# EQUIL data

## EQUIL_rotem_extr1
data 8 -- EQUIL_rotem_extraction 2+3

In [13]:
# X: 'H3PO4_aq'	'SO4_aq'
# T: 'FA_aq'	'solvent_aq'	'H3PO4_org'	'SO4_org'	'water_org'

scaler_data=[[37.23870967741936, 7.199806451612903],
 [21.87534593155935, 6.51727589252697],
 [155.77096774193546,
  0.5283870967741936,
  13.459354838709675,
  1.0374193548387098,
  7.283870967741936],
 [95.42174024805198,
  0.5151599612158911,
  13.221703793258648,
  1.1486934169207217,
  1.7674652479484612]]

scaler_X = preprocessing.StandardScaler().fit([[0,0]])
scaler_T1 = preprocessing.StandardScaler().fit([[0]])
scaler_T2 = preprocessing.StandardScaler().fit([[0]])
scaler_T3 = preprocessing.StandardScaler().fit([[0]])
scaler_T4 = preprocessing.StandardScaler().fit([[0]])

scaler_X.mean_, scaler_X.scale_ = scaler_data[0], scaler_data[1]
scaler_T1.mean_, scaler_T1.scale_ = scaler_data[2][1], scaler_data[3][1] # solvent_aq
scaler_T2.mean_, scaler_T2.scale_ = scaler_data[2][2], scaler_data[3][2] # H3PO4_org
scaler_T3.mean_, scaler_T3.scale_ = scaler_data[2][3], scaler_data[3][3] # SO4_org
scaler_T4.mean_, scaler_T4.scale_ = scaler_data[2][4], scaler_data[3][4] # water_org


# scaler_T1 = preprocessing.StandardScaler().fit([[0]])
# scaler_T2 = preprocessing.StandardScaler().fit([[0]])
# scaler_T3 = preprocessing.StandardScaler().fit([[0]])
# scaler_T3_4 = preprocessing.StandardScaler().fit([[0]])

# scaler_X3.mean_, scaler_X3.scale_ = Xscale[0], Xscale[1]
# scaler_X3_2.mean_, scaler_X3_2.scale_ = [27.03608333,  0.0474    ], [14.68499343,  0.04213688] # מים בקלה חושבו רק על נתונים "טובים
# scaler_T3_1.mean_, scaler_T3_1.scale_=[Tscale[0][0]], [Tscale[1][0]]
# # scaler_T3_2.mean_, scaler_T3_2.scale_=[Tscale[0][1]], [Tscale[1][1]]
# scaler_T3_2.mean_, scaler_T3_2.scale_=[10.81], [1.3091664] # מים בקלה חושבו רק על נתונים "טובים
# scaler_T3_3.mean_, scaler_T3_3.scale_=[Tscale[0][2]], [Tscale[1][2]]
# scaler_T3_4.mean_, scaler_T3_4.scale_=[Tscale[0][3]], [Tscale[1][3]]

In [14]:
class EQUIL_rotem_extr1: #
# data8
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # scaler
        scaler_data=[[37.23870967741936, 7.199806451612903],
        [21.87534593155935, 6.51727589252697],
        [155.77096774193546,
          0.5283870967741936,
          13.459354838709675,
          1.0374193548387098,
          7.283870967741936],
        [95.42174024805198,
          0.5151599612158911,
          13.221703793258648,
          1.1486934169207217,
          1.7674652479484612]]

        scaler_X = preprocessing.StandardScaler().fit([[0,0]])
        scaler_T1 = preprocessing.StandardScaler().fit([[0]]) # solvent_aq
        scaler_T2 = preprocessing.StandardScaler().fit([[0]]) # H3PO4_org
        scaler_T3 = preprocessing.StandardScaler().fit([[0]]) # SO4_org
        scaler_T4 = preprocessing.StandardScaler().fit([[0]]) # water_org

        scaler_X.mean_, scaler_X.scale_ = scaler_data[0], scaler_data[1]
        scaler_T1.mean_, scaler_T1.scale_ = scaler_data[2][1], scaler_data[3][1] # solvent_aq
        scaler_T2.mean_, scaler_T2.scale_ = scaler_data[2][2], scaler_data[3][2] # H3PO4_org
        scaler_T3.mean_, scaler_T3.scale_ = scaler_data[2][3], scaler_data[3][3] # SO4_org
        scaler_T4.mean_, scaler_T4.scale_ = scaler_data[2][4], scaler_data[3][4] # water_org

        X_scaled=scaler_X.transform(x[:,[1,2]])
        X_scaled=np.clip(X_scaled, -2, 2)

        # H3PO4_org
        bp1=VectorBackProp(layers=[2,1,1], hidden_activation = 'sigmoid')
        manual_weights1=[[[-2.2542856480833247], [-0.7115892162097907]],
 [[2.11693061595761]],
 [[-4.9502436032281585]],
 [[3.912054301757645]]]
        bp1.import_weights(manual_weights1)
        predics1=scaler_T2.inverse_transform(bp1.run(X_scaled))

        # water_org
        bp2=VectorBackProp(layers=[2,2,1], hidden_activation = 'sigmoid')
        manual_weights2=[[[3.865511470369896, 8.917531695210526],
  [2.5196000386044317, 1.9258172086316168]],
 [[-2.2766428706488764, -1.5079705074058127]],
 [[2.7771600466732367], [0.9281118411212552]],
 [[-1.1710007634929602]]]
        bp2.import_weights(manual_weights2)
        predics2=scaler_T4.inverse_transform(bp2.run(X_scaled))

        # solvent_aq
        bp3=VectorBackProp(layers=[2,1,1], hidden_activation = 'linear')
        manual_weights3=[[[0], [0]], [[0]], [[0]], [[0.9]]]
        bp3.import_weights(manual_weights3)
        predics3=scaler_T1.inverse_transform(bp3.run(X_scaled))

        # SO4_org
        bp4=VectorBackProp(layers=[2,3,1], hidden_activation = 'sigmoid')
        manual_weights4=[[[5.255129630698163, 2.5370572590919305, -2.771603241282131],
  [4.30266945368854, 1.6643297587562575, 2.759812469181559]],
 [[-6.720893024720382, -0.7893443670545283, 5.835081175805133]],
 [[5.160798955335814], [2.555891510916236], [2.30456660012357]],
 [[-3.1529870476605937]]]
        bp4.import_weights(manual_weights4)
        predics4=scaler_T3.inverse_transform(bp4.run(X_scaled))

        self.xout = np.array([predics3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics2[:,0], predics1[:,0], predics4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

In [15]:
scaler_X = preprocessing.StandardScaler().fit([[0,0]])
scaler_X.mean_, scaler_X.scale_ = scaler_data[0], scaler_data[1]
scaler_X.mean_

[37.23870967741936, 7.199806451612903]

In [16]:
# @title class Stage

class Stage:
    """Attributes:
        Oin:                  Organic phase mass inlet flow.
        Ain:                  Aqueous phase mass inlet flow.
        yin:                  List of organic phase inlet concentrations. [%water_org, %H3PO4_org, ...]
        xin:                  List of aqueous phase inlet concentrations. [%solvent_aq, %H3PO4_aq, ...]
        Oout:                 Organic phase mass outlet flow.
        Aout:                 Aqueous phase mass outlet flow.
        yout_tag:             List of organic phase equilibrium concentrations.
        yout:                 List of organic phase outlet concentrations.
        xout:                 List of aqueous phase outlet concentrations.
        runs:                 Number of iterations.
        eff:                  Stage efficiency (0.0-1.0).
        errors:               Numpy array of errors.
        conv:                 Numpy array of convergence.
        EQUIL:                Object class for calculating equilibrium concentrations.
        entrainment_perc:     Percentage of entrainments [org_in, aq_in, org_out, aq_out]. Additionally to the main stream: Oin 100 kg with 5% entrainment means 100 kg organic and 5 kg aqueous -- 105 overall.
        entrainment_comp_in:  Composition of inlet entrainments in [aqueous phase in Oin, organic phase in Ain] --> [[%solvent_aq, %H3PO4_aq, ...], [%water_org, %H3PO4_org, ...], ]. Outlet compositions are calculated: yout, xout.
        entrainment_out:      List of entrainment out data: [entrainment_perc[2:4], yout, xout].

        Functions:
        output():       Print output summary.
        roundoutput():  Print rounded output summary."""

    def __init__(self, Oin, Ain, yin, xin, EQUIL, eff=1., entrainment_perc=0, entrainment_comp_in=0, convergence=1E-3):

        Oin_clear = float(Oin)
        Ain_clear = float(Ain)
        self.yin = np.array(yin, dtype=float) # [%water_org, %H3PO4_org, ...]
        self.xin = np.array(xin, dtype=float) # [%solvent_aq, %H3PO4_aq, ...]
        self.EQUIL = EQUIL
        self.eff = eff
        self.entrainment_perc = entrainment_perc
        self.entrainment_comp_in = entrainment_comp_in
        self.check_inputs()
        Oin = Oin_clear + Ain_clear*self.entrainment_perc[1]/100.
        Ain = Ain_clear + Oin_clear*self.entrainment_perc[0]/100.
        a = Oin_clear*self.yin/100. + Ain_clear*self.entrainment_perc[1]/100*self.entrainment_comp_in[1]/100. # [%water_org, %H3PO4_org, %SO4_org, ...]
        b = Ain_clear*self.xin/100. + Oin_clear*self.entrainment_perc[0]/100*self.entrainment_comp_in[0]/100. # [%solvent_aq, %H3PO4_aq, ...]

        #k=0
        c_k=(a+b)/2
        d=a+b-c_k
        alfa=0 # d[0]=alfa*Aout
        A=np.array([[1,1,0],[-1,0,1],[1,0,-alfa]]) # d[0],Oout,Aout
        B=np.array([[Oin+sum(c_k)-sum(a)+b[0]],
                    [Ain-sum(b)+a[0]-c_k[0]+sum(d)-d[0]],
                    [0]])
        C=np.dot(np.linalg.inv(A),B)
        d[0],Oout,Aout = C[0,0],C[1,0],C[2,0]
        xout=d/Aout*100 #
        out_data = self.EQUIL([xout])
        xout=out_data.xout[0]
        yout_tag=out_data.yout[0]
        yout=self.yin+self.eff*(yout_tag-self.yin)
        d=Aout*xout/100
        alfa=d[0]/Aout
        e_k=c_k-Oout*yout/100
        errors=np.array([e_k/(Oout*yout/100+1E-7)]) #

        #k=1
        c_k1=Oout*yout/100
        conv=[np.ones(len(c_k1))]
        d=a+b-c_k1
        A=np.array([[1,1,0],[-1,0,1],[1,0,-alfa]]) # d[0],Oout,Aout
        B=np.array([[Oin+sum(c_k1)-sum(a)+b[0]],
                    [Ain-sum(b)+a[0]-c_k1[0]+sum(d)-d[0]],
                    [0]])
        C=np.dot(np.linalg.inv(A),B)
        d[0],Oout,Aout = C[0,0],C[1,0],C[2,0]
        xout=d/Aout*100
        out_data = self.EQUIL([xout])
        xout=out_data.xout[0]
        yout_tag=out_data.yout[0]
        yout=self.yin+self.eff*(yout_tag-self.yin)
        d=Aout*xout/100
        alfa=d[0]/Aout
        e_k1=c_k1-Oout*yout/100
        errors=np.concatenate((errors, [e_k1/(Oout*yout/100+1E-7)]), axis=0)

        #k=2
        c_k2=c_k1-e_k1/((e_k1-e_k+1E-7)/(c_k1-c_k+1E-7))
        conv=np.concatenate((conv, [(c_k2-c_k1)/(c_k2+1E-7)]), axis=0)
        d=a+b-c_k2
        A=np.array([[1,1,0],[-1,0,1],[1,0,-alfa]]) # d[0],Oout,Aout
        B=np.array([[Oin+sum(c_k2)-sum(a)+b[0]],
                    [Ain-sum(b)+a[0]-c_k2[0]+sum(d)-d[0]],
                    [0]])
        C=np.dot(np.linalg.inv(A),B)
        d[0],Oout,Aout = C[0,0],C[1,0],C[2,0]
        xout=d/Aout*100
        out_data = self.EQUIL([xout])
        xout=out_data.xout[0]
        yout_tag=out_data.yout[0]
        yout=self.yin+self.eff*(yout_tag-self.yin)
        d=Aout*xout/100
        alfa=d[0]/Aout
        e_k2=c_k2-Oout*yout/100
        errors=np.concatenate((errors, [e_k2/(Oout*yout/100+1E-7)]), axis=0)

        #k>2
        i=0
        while max(abs(conv[-1]))>convergence:
            c_k, e_k=c_k1, e_k1
            c_k1, e_k1=c_k2, e_k2
            c_k2=c_k1-e_k1/((e_k1-e_k+1E-7)/(c_k1-c_k+1E-7))
            conv=np.concatenate((conv, [(c_k2-c_k1)/(c_k2+1E-7)]), axis=0)
            d=a+b-c_k2
            A=np.array([[1,1,0],[-1,0,1],[1,0,-alfa]]) # d[0],Oout,Aout
            B=np.array([[Oin+sum(c_k2)-sum(a)+b[0]],
                        [Ain-sum(b)+a[0]-c_k2[0]+sum(d)-d[0]],
                        [0]])
            C=np.dot(np.linalg.inv(A),B)
            d[0],Oout,Aout = C[0,0],C[1,0],C[2,0]
            Oout=Oin+sum(c_k2)-sum(a)+b[0]-d[0]
            Aout=Ain+sum(d)-sum(b)+a[0]-c_k2[0]
            Aout=abs(Aout)
            xout=d/Aout*100
            out_data = self.EQUIL([xout])
            xout=out_data.xout[0]
            yout_tag=out_data.yout[0]
            yout=self.yin+self.eff*(yout_tag-self.yin)
            d=Aout*xout/100
            alfa=d[0]/Aout
            e_k2=c_k2-Oout*yout/100
            errors=np.concatenate((errors, [e_k2/(Oout*yout/100+1E-7)]), axis=0)
            i+=1
            if i==200:
                break

        self.errors=errors
        self.conv=conv
        self.runs=i
        self.Oin=Oin
        self.Ain=Ain
        Oout_clean=Oout
        Aout_clean=Aout
        A=np.array([[1,self.entrainment_perc[3]/100],[self.entrainment_perc[2]/100,1]])
        B=np.array([[Oout_clean],[Aout_clean]])
        C=np.dot(np.linalg.inv(A),B)
        self.Oout, self.Aout = C[0,0], C[1,0]
        self.xout=xout
        self.yout_tag=yout_tag
        self.yout=yout
        self.entrainment_out=[self.entrainment_perc[2:4], self.yout, self.xout]
        # self.K=self.xout/self.yout

    def check_inputs(self):
        if np.isscalar(self.eff):
            self.eff = np.ones(len(self.yin)) * self.eff
            self.eff[0] = 1. # water and solvent
        else:
            if len(self.eff) != len(self.yin):
                print ('Length of efficiency array is not correct.')
            self.eff = np.ones(len(self.yin))* self.eff
            self.eff[0] = 1.

        if np.isscalar(self.entrainment_perc):
            self.entrainment_perc = np.ones(4) * self.entrainment_perc
        else:
            if len(self.entrainment_perc) != 4:
                print ('List of entrainment percents should consist of 4 elements [org_in, aq_in, org_out, aq_out] or single number.')
            self.entrainment_perc = np.ones(4) * self.entrainment_perc

        if np.isscalar(self.entrainment_comp_in):
            self.entrainment_comp_in = np.ones([2,len(self.yin)]) * self.entrainment_comp_in
        else:
            if len(self.entrainment_comp_in[0]) != len(self.yin):
                print ('Array of entrainment compositions is not correct.')
            self.entrainment_comp_in = np.ones(len(self.yin)) * self.entrainment_comp_in

    def output(self):
        print ("Oout=", self.Oout)
        print ("yout=", self.yout)
        print ("yout_tag=", self.yout_tag, " %water_org_equilib, %solubles_equilib")
        print ("Aout=", self.Aout)
        print ("xout=", self.xout)
        print ("equilibrium efficiency=", self.eff)
        print ("error=", self.errors[-1])
        print ("convergence=", self.conv[-1])
        # print ("K=", self.K)

    def roundoutput(self):
        print ("Oout=", round(self.Oout,1))
        print ("yout=", [round(num,3) for num in self.yout], " %water_org, %solubles")
        print ("yout_tag=", [round(num,3) for num in self.yout_tag], " %water_org_equilib, %solubles_equilib")
        print ("Aout=", round(self.Aout,2))
        print ("xout=", [round(num,3) for num in self.xout], " %solvent_aq, %solubles")
        print ("equilibrium efficiency=", self.eff)
        print ("error=", [round(num,7) for num in self.errors[-1]])
        print ("convergence=", [round(num,7) for num in self.conv[-1]])
        # print ("K=", [round(num,5) for num in self.K])


In [58]:
# @title class BatteryConv

class BatteryConv:
    """Attributes:
        stages_num:                       Number of stages.
        stages:                           Array of Stage objects (0 -- stage_num-1). Organic inlet to stages[0] -- Stage 1.
        Oin/Ain/Oin/Oout:                 Organic/aqueous phase mass inlet/outlet flow.
        Oin/Ain/Oin/Oout_list:            List of organic/aqueous phase mass inlet/outlet flows for stages.
        yin/xin/yout_tag/yout/xout_list:  List of lists of organic/aqueous phase inlet/outlet concentrations: [stage, component]
        runs:                             Number of iterations.
        eff_list:                         List of stage efficiencies (0.0-1.0).
        errors:                           Numpy array of errors.
        EQUIL:                            Object class for calculating equilibrium concentrations.
        K:                                List of distribute ratios for solubles. %aq/%org
        entrainment_perc_in:              Entrainment percent for inlets [org_in, aq_in].
        entrainment_perc_out:             Entrainment percent data should consist of 2D array (stages_num, 2)  or 1D array [org_out, aq_out] or single number.
        entrainment_perc_list:            Entrainment percent data arranged for stages.
        entrainment_comp_in:              Composition of inlet entrainments in [aqueous phase in Oin, organic phase in Ain] --> [[%solvent_aq, %H3PO4_aq, ...], [%water_org, %H3PO4_org, ...]]. Outlet compositions are calculated: yout, xout.
        entrainment_comp_list:            List of entrainment compositions [[%solvent_aq, %H3PO4_aq, ...],[%water_org, %H3PO4_org, ...], ... stage_num].
        entrainment_comp_flat:            List of entrainment compositions arranged for presentation in table.
        """

    def __init__(self, stages_num, Oin, Ain, yin, xin, EQUIL, method='successive', eff=1., max_iter=100, entrainment_perc_in=0, entrainment_comp_in=0, entrainment_perc_out=0, convergence=1E-4):
        self.stages_num = stages_num
        self.EQUIL = EQUIL
        self.eff = eff
        self.entrainment_perc_in = entrainment_perc_in
        self.entrainment_comp_in = entrainment_comp_in
        self.entrainment_perc_out = entrainment_perc_out

        self.initialize_arrays(Oin, Ain, yin, xin)
        self.check_inputs()

        if self.stages_num==1:
            self.solve_battery()
            self.runs=0
            self.finalize()
            return

        # Initial run for error calculation
        self.solve_battery() # Iterates over stages and runs Stage for each one
        self.update_outs()
        self.update_ins()
        self.update_errors()
        self.get_abs_errors() # Compares in data and out data of adjacent stages
        self.get_abs_errors2() # Compares in data and out data of adjacent stages
        # self.errors=np.concatenate((self.errors,[(self.xout_list[1]-self.xin_list[0])/(self.xout_list[1]+1E-7)]), axis=0)

        if method=='successive':
            self.successive_substitution(max_iter=max_iter, convergence=convergence)
        if method=='secant':
            self.secant(max_iter=max_iter, convergence=convergence)

    def secant(self, max_iter=100, convergence=1E-4):
        j=0
        while abs(self.errors_array.iloc[-1].any()) > convergence:
        # for j in range(20):
            new_ins = self.outs_array.iloc[-1] - self.errors_array.iloc[-1] * (self.outs_array.iloc[-1] - self.outs_array.iloc[-2]) / (self.errors_array.iloc[-1] - self.errors_array.iloc[-2] + 1e-10)
            self.update_stage_connections_from_array(new_ins)
            self.solve_battery() # Iterates over stages and runs Stage for each one
            self.update_outs()
            self.update_ins()
            self.update_errors()
            j+=1
            if j==max_iter:
                print (max_iter, 'iterations!')
                break
        self.runs = j

    def successive_substitution(self, max_iter=100, convergence=1E-4):
        j=0
        # while max(abs(self.errors[-1]))>convergence or max(abs(self.errors[-2]))>convergence:
        while abs(self.errors_array.iloc[-1].any()) > convergence:
        # for j in range(20):
            self.update_stage_connections() # Replaces in data by out data of adjacent stage
            self.solve_battery() # Iterates over stages and runs Stage for each one
            self.update_outs()
            self.update_ins()
            self.update_errors()
            self.get_abs_errors() # Compares in data and out data of adjacent stages
            self.get_abs_errors2() # Compares in data and out data of adjacent stages
            # self.errors=np.concatenate((self.errors,[(self.xout_list[1]-self.xin_list[0])/(self.xout_list[1]+1E-7)]), axis=0)

            j+=1
            if j==max_iter:
                print (max_iter, 'iterations!')
                break
        self.runs = j

        self.finalize()

    def initialize_arrays(self, Oin, Ain, yin, xin):
        self.Oin_list = np.ones(self.stages_num) * Oin
        self.Ain_list = np.ones(self.stages_num) * Ain
        yin = np.array(yin, dtype=float) # [%water_org, %H3PO4_org, ...]
        xin = np.array(xin, dtype=float) # [%solvent_aq, %H3PO4_aq, ...]
        self.yin_list = np.array([yin for i in range(self.stages_num)])
        self.xin_list = np.array([xin for i in range(self.stages_num)])
        self.Oout_list = np.zeros(self.stages_num)
        self.Aout_list = np.zeros(self.stages_num)
        self.yout_tag_list = np.zeros((self.stages_num, len(yin)))
        self.yout_list = np.zeros((self.stages_num, len(yin)))
        self.xout_list = np.zeros((self.stages_num, len(xin)))
        self.stages = [[] for i in range(self.stages_num)] # List of stages
        # self.errors=np.zeros([2,len(xin)]) # for stages_num=1
        self.error_list = pd.DataFrame()
        self.error_list2 = pd.DataFrame()
        self.outs_array = pd.DataFrame(np.zeros((1,(self.stages_num-1)*(len(xin)*2+2))))
        self.ins_array = pd.DataFrame()
        self.errors_array = pd.DataFrame(np.zeros((1,(self.stages_num-1)*(len(xin)*2+2))))

    def check_inputs(self):
        # --------
        eff, stages_num, yin_len = self.eff, self.stages_num, len(self.yin_list[0])
        if np.isscalar(eff):                                             # scalar
            eff_out = np.ones([stages_num, yin_len]) * eff
        else:
            eff = np.array(eff)
            eff_out = np.ones([stages_num, yin_len])
            if np.isscalar(eff[0]):                                      # vector for elements [water, P2O5, ...] same for all stages
                a = np.array(list(map(lambda x:eff[:yin_len], eff_out))) # 2D array (stage_num, yin_len) with rows of eff
                eff_out[:a.shape[0], :a.shape[1]]=a                      # for the case if eff is shorter than yin_len
            else:
                if eff.shape[1] == 1:                                    # vector for stages same for all elements
                    b = eff.T[0]
                    a = np.array(list(map(lambda x:b[:stages_num], eff_out.T)))
                    eff_out.T[:a.shape[0], :a.shape[1]]=a
                if eff.shape[1] > 1:                                     # list of lists
                    eff_out[:eff.shape[0], :eff.shape[1]]=eff            # for the case if eff is shorter than yin_len
        self.eff_list = eff_out
        self.eff_list[:,0]=1
        # ----------
        epi = self.entrainment_perc_in
        if np.isscalar(epi):
            epi = np.ones(2) * epi
        else:
            epi = np.array(epi, dtype=float)

        epo = self.entrainment_perc_out
        if np.isscalar(epo):
            epo_out = np.ones([stages_num, 2]) * epo
        else:
            epo = np.array(epo, dtype=float)
            epo_out = np.ones([stages_num, 2])
            if np.isscalar(epo[0]): # same [org_out, aq_out] for all stages
                if len(epo) != 2:
                    print ('Entrainment percent data should consist of 2D array (stages_num, 2)  or 1D array [org_out, aq_out] or single number.')
                epo_out = epo_out * epo
            else:
                if epo.shape != (stages_num,2):
                    print ('Entrainment percent data should consist of 2D array (stages_num, 2)  or 1D array [org_out, aq_out] or single number.')
                epo_out = epo

        ep1 = np.concatenate([[epi], epo_out, [epi]], axis=0)
        ep2 = [[ep1[i,0],ep1[i+2,1]] for i in range(stages_num)]
        self.entrainment_perc_list = np.concatenate([ep2,epo_out], axis=1)
        # ------------
        if np.isscalar(self.entrainment_comp_in):
            self.entrainment_comp_in = np.ones([2, len(self.yin_list[0])]) * self.entrainment_comp_in
        else:
            if len(self.entrainment_comp_in[0]) != len(self.yin_list[0]):
                print ('Array of entrainment compositions is not correct.')
        self.entrainment_comp_list = np.array([self.entrainment_comp_in for i in range(self.stages_num)])*1.

    def update_stage_connections(self):
        """Update connections between stages - flows and compositions."""
        for i in range(1, self.stages_num):
            self.Oin_list[i] = self.Oout_list[i-1] # Update organic phase connections (forward direction)
            self.yin_list[i] = self.yout_list[i-1]
            self.entrainment_comp_list[i-1][1] = self.yout_list[i]
            self.Ain_list[i-1] = self.Aout_list[i] # Update aqueous phase connections (backward direction)
            self.xin_list[i-1] = self.xout_list[i]
            self.entrainment_comp_list[i][0] = self.xout_list[i-1]

    def update_stage_connections_from_array(self, new_ins):
        """Update connections between stages - flows and compositions -
        from the Series new_ins."""
        n = len(self.xin_list[0])
        half = int(len(new_ins)/2)
        for i in range(0, self.stages_num-1):
            self.Oin_list[i+1] = new_ins[i*(n+1)]
            self.yin_list[i+1] = new_ins[i*(n+1)+1:i*(n+1)+n+1].to_numpy()
            self.Ain_list[i] = new_ins[half+i*(n+1)]
            self.xin_list[i] = new_ins[half+i*(n+1)+1:half+i*(n+1)+n+1].to_numpy()
            self.entrainment_comp_list[i][1] = self.yout_list[i+1]
            self.entrainment_comp_list[i+1][0] = self.xout_list[i]

    def get_abs_errors2(self):
        """Calculate errors."""
        err=[]
        for i in range(1, self.stages_num):
            a=np.array([self.Oin_list[i] - self.Oout_list[i-1]])
            b=self.yin_list[i] - self.yout_list[i-1]
            c=np.array([self.Ain_list[i-1] - self.Aout_list[i]])
            d=self.xin_list[i-1] - self.xout_list[i]
            err.extend([a.item(), *b.tolist(), c.item(), *d.tolist()])
        err = np.array(err).reshape(1,-1)
        self.error_list2 = pd.concat([self.error_list2, pd.DataFrame(err)], axis=0, ignore_index=True)

    def get_abs_errors(self):
        """Calculate errors."""
        for i in range(1, self.stages_num):
            a=np.array([self.Oin_list[i] - self.Oout_list[i-1]])
            b=self.yin_list[i] - self.yout_list[i-1]
            c=np.array([self.Ain_list[i-1] - self.Aout_list[i]])
            d=self.xin_list[i-1] - self.xout_list[i]
        err = np.concatenate([a,b,c,d]).reshape(1,-1)
        self.error_list = pd.concat([self.error_list, pd.DataFrame(err)], axis=0, ignore_index=True)

    def get_rel_errors(self):
        """Calculate errors."""
        for i in range(1, self.stages_num): # Organic phase (forward direction)
            a=np.array([(self.Oin_list[i] - self.Oout_list[i-1]) / (self.Oin_list[i] + 1e-7)])
            b=(self.yin_list[i] - self.yout_list[i-1]) / (self.yin_list[i] + 1e-7)
        for i in range(0, self.stages_num-1): # Aqueous phase (backward direction)
            c=np.array([(self.Ain_list[i] - self.Aout_list[i+1]) / (self.Ain_list[i] + 1e-7)])
            d=(self.xin_list[i] - self.xout_list[i+1])/ (self.xin_list[i] + 1e-7)
        err = np.concatenate([a,b,c,d]).reshape(1,-1)
        self.error_list = pd.concat([self.error_list, pd.DataFrame(err)], axis=0, ignore_index=True)

    def update_outs(self):
        outs=[]
        for i in range(1, self.stages_num): # Organic phase (forward direction)
            a=np.array([self.Oout_list[i-1]])
            b=self.yout_list[i-1]
            outs.extend([a.item(), *b.tolist()])
        for i in range(0, self.stages_num-1): # Aqueous phase (backward direction)
            c=np.array([self.Aout_list[i+1]])
            d=self.xout_list[i+1]
            outs.extend([c.item(), *d.tolist()])
        outs = np.array(outs).reshape(1,-1)
        self.outs_array=pd.concat([self.outs_array, pd.DataFrame(outs)], axis=0, ignore_index=True)

    def update_ins(self):
        ins=[]
        for i in range(1, self.stages_num): # Organic phase (forward direction)
            a=np.array([self.Oin_list[i]])
            b=self.yin_list[i]
            ins.extend([a.item(), *b.tolist()])
        for i in range(0, self.stages_num-1): # Aqueous phase (backward direction)
            c=np.array([self.Ain_list[i]])
            d=self.xin_list[i]
            ins.extend([c.item(), *d.tolist()])
        ins = np.array(ins).reshape(1,-1)
        self.ins_array=pd.concat([self.ins_array, pd.DataFrame(ins)], axis=0, ignore_index=True)

    def update_errors(self):
        """Calculate errors."""
        new_error = self.outs_array.iloc[-1] - self.ins_array.iloc[-1]
        self.errors_array=pd.concat([self.errors_array, new_error.to_frame().T], ignore_index=True)

    def calculate_stage(self, stage_idx, flows_comps):
        """Calculate single stage equilibrium and update flows/compositions."""
        Oin = flows_comps[0]
        yin = flows_comps[1]
        Ain = flows_comps[2]
        xin = flows_comps[3]
        stage = Stage(
            Oin=Oin,
            Ain=Ain,
            yin=yin,
            xin=xin,
            EQUIL=self.EQUIL,
            eff=self.eff_list[stage_idx],
            entrainment_perc=self.entrainment_perc_list[stage_idx],
            entrainment_comp_in=self.entrainment_comp_list[stage_idx]
        )
        return [stage.Oout, stage.yout, stage.yout_tag, stage.Aout, stage.xout]

    def solve_battery(self):
        for i in range(self.stages_num):
            flows_comps = [self.Oin_list[i],self.yin_list[i],self.Ain_list[i],self.xin_list[i]]
            results = self.calculate_stage(i, flows_comps)
            self.Oout_list[i] = results[0]
            self.yout_list[i] = results[1]
            self.yout_tag_list[i] = results[2]
            self.Aout_list[i] = results[3]
            self.xout_list[i] = results[4]

    def finalize(self):
        self.K = self.xout_list/(self.yout_tag_list+1E-7)
        self.Oout, self.Aout, self.yout_tag, self.yout, self.xout, = self.Oout_list[-1], self.Aout_list[0], self.yout_tag_list[-1], self.yout_list[-1], self.xout_list[0],
        entrainment_comp_flat = []
        for stage in range(self.stages_num):
            entrainment_comp_flat.append([i for item in self.entrainment_comp_list[stage] for i in item])
        self.entrainment_comp_flat = np.array(entrainment_comp_flat)

In [18]:
class BatteryTable:
    """
    """
    def __init__(self, batt, show=True, index_names=[]):
        len1=len(batt.xin_list[0])
        out=np.concatenate((
            [batt.Oin_list],
            batt.yin_list.T,
            [batt.Ain_list],
            batt.xin_list.T,
            [batt.Oout_list],
            batt.yout_list.T,
            [batt.Aout_list],
            batt.xout_list.T,
            ),axis=0)
        Name = (
            ['Org in, ton/hr']+
            ['y'+str(a+1)+' in, %' for a in range(len1)]+
            ['Aq in, ton/hr']+
            ['x'+str(a+1)+' in, %' for a in range(len1)]+
            ['Org out, ton/hr']+
            ['y'+str(a+1)+' out, %' for a in range(len1)]+
            ['Aq out, ton/hr']+
            ['x'+str(a+1)+' out, %' for a in range(len1)]
            )
        columns = ['Stage '+str(a) for a in range(1,batt.stages_num+1)]
        if len(index_names)>0:
            Name=index_names
        self.data=pd.DataFrame(out, columns=columns, index=Name)
        if show:
            display (self.data)

class BatteryTableFull:
    """
    """
    def __init__(self, batt, show=True, index_names=[]):
        len1=len(batt.xin_list[0])
        out=np.concatenate((
            [batt.Oin_list],
            batt.yin_list.T,
            [batt.Ain_list],
            batt.xin_list.T,
            [batt.Oout_list],
            batt.yout_list.T,
            [batt.Aout_list],
            batt.xout_list.T,
            batt.K.T[1:],
            batt.entrainment_perc_list.T,
            np.concatenate([[batt.Oin_list],[batt.Ain_list],[batt.Oout_list],[batt.Aout_list]], axis=0)*batt.entrainment_perc_list.T/100,
            batt.entrainment_comp_flat.T,
            ),axis=0)
        Name = (
            ['Org in, ton/hr']+
            ['y'+str(a+1)+' in, %' for a in range(len1)]+
            ['Aq in, ton/hr']+
            ['x'+str(a+1)+' in, %' for a in range(len1)]+
            ['Org out, ton/hr']+
            ['y'+str(a+1)+' out, %' for a in range(len1)]+
            ['Aq out, ton/hr']+
            ['x'+str(a+1)+' out, %' for a in range(len1)]+
            ['K'+str(a+1)+' [aq/org]' for a in range(1,len1)]+
            ['aq in org_in, %']+['org in aq_in, %']+['aq in org_out, %']+['org in aq_out, %']+
            ['aq in org_in, ton/hr']+['org in aq_in, ton/hr']+['aq in org_out, ton/hr']+['org in aq_out, ton/hr']+
            ['aq contrain in Oin - x'+str(a+1)+'%' for a in range(len1)]+
            ['org contrain in Ain - y'+str(a+1)+'%' for a in range(len1)]
            )
        columns = ['Stage '+str(a) for a in range(1,batt.stages_num+1)]
        if len(index_names)>0:
            Name=index_names
        self.data=pd.DataFrame(out, columns=columns, index=Name)
        if show:
            display (self.data)


# Cases

## Extraction 1

### test

In [19]:
x=np.array([[0,0,0],[100,100,100]])

In [20]:
X_scaled=scaler_X.transform(x[:,[1,2]])
X_scaled

array([[-1.70231409, -1.10472636],
       [ 2.86904219, 14.23910773]])

In [21]:
np.clip(X_scaled, -1, 1)

array([[-1., -1.],
       [ 1.,  1.]])

In [22]:
EQUIL_rotem_extr1([[0,30,9]]).xout # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]

array([[ 0.99203106, 30.        ,  9.        ]])

In [23]:
EQUIL_rotem_extr1([[0,30,9]]).yout # [[%water_org, %H3PO4_org, %SO4_org], ...]

array([[5.51179427, 3.98631405, 0.76135935]])

In [24]:
test1 = LLC.Stage(173.2, 15, [8.2, .06, .06], [1, 30, 15], EQUIL_rotem_extr1, eff=.8, convergence=1E-4)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]])
test1.runs

12

In [25]:
test1.output()

Oout= 169.57346101275468
yout= [5.25893594 0.85771993 0.26318594]
yout_tag= [5.25893594 1.05714991 0.31398242]  %water_org_equilib, %solubles_equilib
Aout= 18.626538987245322
xout= [ 0.99203106 16.90836857 10.24168195]
equilibrium efficiency= [1.  0.8 0.8]
error= [-3.23113626e-07  7.40483595e-06 -9.94591222e-05]
convergence= [-3.11837121e-06 -7.43013235e-05  1.02388794e-05]


In [26]:
test2 = LLC.Stage(173.2, 50, [6, 6, 1], [1, 60, .1], EQUIL_rotem_extr1, eff=.8, convergence=1E-4)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]])
test2.runs

61

In [27]:
test2.output()

Oout= 188.48554843890858
yout= [ 7.27413364 12.0435486   0.695869  ]
yout_tag= [ 7.27413364 13.55443574  0.61983625]  %water_org_equilib, %solubles_equilib
Aout= 34.7144515610914
xout= [ 0.99203106 51.17669027  1.36347826]
equilibrium efficiency= [1.  0.8 0.8]
error= [ 1.94927193e-05 -3.26232763e-03 -2.23884450e-03]
convergence= [-4.70070399e-05  3.24166294e-05  8.06081543e-05]


In [28]:
# test3 = LLC52.Stage(173.2, 50, [6, 6, .5], [1, 60, .1], EQUIL_rotem_extr1, eff=.8, convergence=1E-4)

### mesh

In [29]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 70, 31), np.linspace(0, 25, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh3 = x_mesh.flatten().reshape(31*31,1)
y_mesh3 = y_mesh.flatten().reshape(31*31,1)
mesh3=np.concatenate((x_mesh3, y_mesh3), axis=1)
# x_mesh[:5]

In [30]:
a=np.concatenate((np.zeros([len(mesh3),1]),mesh3), axis=1)

In [31]:
mesh_predict_x=np.array([EQUIL_rotem_extr1([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_extr1([row]).yout for row in a])
mesh_predics1_1=mesh_predict_y[:,:,1]
mesh_predics1_2=mesh_predict_y[:,:,0]
mesh_predics1_3=mesh_predict_x[:,:,0]
mesh_predics1_4=mesh_predict_y[:,:,2]

### 1 - Extraction1 - raffinate old Battery

In [32]:
# bat1_1=LLC.Battery(stages_num=3, Oin=173.2, Ain=15, yin=[8.2, .06, .06], xin=[1, 30, 15], EQUIL=EQUIL_rotem_extr1, eff=.8, max_iter=200)#, convergence=1E-4)
bat1_1=LLC.Battery(stages_num=3, Oin=173.2, Ain=50, yin=[6, 6, 1 ], xin=[1, 60, .1], EQUIL=EQUIL_rotem_extr1, eff=.8, max_iter=10, convergence=1E-4)
bat1_1.runs

10 iterations!


10

In [33]:
table1_1=LLC.BatteryTable(bat1_1).data

,Stage 1,Stage 2,Stage 3
"Org in, ton/hr",173.200000,176.687477,184.008921
"y1 in, %",6.000000,5.743703,6.830781
"y2 in, %",6.000000,6.156989,9.791760
"y3 in, %",1.000000,0.613717,0.568690
"Aq in, ton/hr",31.828082,39.597221,50.000000
"x1 in, %",0.992031,0.992031,1.000000
"x2 in, %",48.171619,53.425041,60.000000
"x3 in, %",1.410934,0.695291,0.100000
"Org out, ton/hr",176.202503,185.375811,195.000681
"y1 out, %",6.222004,6.744944,7.370456


In [34]:
# @markdown plots
fig = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

# visib='legendonly'
visib=True
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat1_1
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01), aspectratio=dict(x=1, y=1, z=1))
fig.update_scenes(camera_projection_type="orthographic")

fig.show()

### 6 - Extraction1 - raffinate

In [63]:
bat1_2=BatteryConv(stages_num=3, Oin=173.2, Ain=50, yin=[6, 6, 1 ], xin=[1, 60, .1], EQUIL=EQUIL_rotem_extr1, method='secant', eff=.8, max_iter=50, convergence=1E-4)#, entrainment_perc_in=[1,2], entrainment_comp_in=[[10,10,10],[5,5,5]], entrainment_perc_out=[3,4])
bat1_2.runs

50 iterations!


50

In [39]:
bat1_2.outs_array

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,188.337411,7.261250,11.974000,0.686887,188.337411,7.261250,11.974000,0.686887,3.486259e+01,0.992031,51.130941,1.333460,34.862589,0.992031,51.130941,1.333460
2,171.035903,6.134659,7.250323,0.940735,0.000001,5.214212,-0.149153,0.051183,3.004589e-07,0.992031,0.676753,-0.225322,49.994441,0.992031,60.004560,0.099992
3,174.215741,6.782821,6.035981,1.653280,183.989660,9.058370,30.707455,0.589112,4.440710e+01,0.992031,63.065567,0.756887,49.994436,0.992031,60.004560,0.099989
4,174.738934,7.264712,7.383000,1.822198,183.830832,10.918807,15.425309,9.558008,4.011041e+01,0.992031,34.414422,36.999095,49.994441,0.992031,60.004559,0.099993
5,172.798800,11.762971,52.217477,7.334066,185.741032,7.448956,12.733439,1.085216,3.896737e+01,0.992031,50.409034,2.519903,49.994279,0.992031,60.004340,0.099988
6,170.342371,7.697679,5.737702,2.466380,191.610555,5.323839,3.460542,1.566831,3.255515e+01,0.992031,27.241719,7.706245,50.016066,0.992031,60.022058,0.100101
7,177.492559,9.676783,16.557804,2.202411,193.638739,5.280476,3.147152,1.154636,3.161275e+01,0.992031,27.057017,6.456119,48.912826,0.992031,59.786542,0.093504
8,-7301.181579,5.214211,1.048092,0.251371,187.748137,7.463061,12.590080,1.151587,3.710310e+01,0.992031,50.075076,2.763216,5.121498,0.992031,57.167401,3.587495
9,176.376026,8.422507,11.778227,1.779715,191.216103,6.904028,11.456532,-2.518567,3.403527e+01,0.992031,57.903145,-11.451269,47.844970,0.992031,47.709988,1.824671


In [65]:
bat1_2.errors_array

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,15.137411,1.261250,5.974000,-0.313113,1.513741e+01,1.261250,5.974000,-0.313113,-1.513741e+01,-0.007969,-8.869059,1.233460,-15.137411,-0.007969,-8.869059,1.233460
2,174.385457,7.693019,5.728623,2.462474,6.997394e-11,5.214210,-0.154719,0.051513,-6.997394e-11,0.992031,-0.674633,0.224838,49.994441,0.992031,60.004560,0.099992
3,-17.740718,-1.406966,145.860988,0.061994,3.598668e+02,-2.700203,-28.346869,0.087303,1.318238e+02,0.000000,-104.419496,0.672234,11.615026,0.000000,7.730938,0.108811
4,2.067236,0.724706,2.684196,0.394486,1.872313e+02,1.935636,11.267131,0.823206,3.491339e+01,0.000000,50.496234,1.186444,0.000009,0.000000,-0.000003,-0.000031
5,1.083370,1.050220,0.309839,0.847056,1.870253e+02,1.049469,13.488885,0.445823,3.694720e+01,0.000000,54.121983,1.162313,-0.000007,0.000000,0.000005,-0.000009
6,-0.968943,2.255918,0.952002,0.574929,1.939928e+02,-0.147059,14.837941,0.032388,3.199232e+01,0.000000,30.915180,4.929567,-0.000005,0.000000,0.000012,-0.000007
7,-0.747404,-0.703528,-2.353339,0.631516,1.835584e+02,-0.202834,10.064187,0.039401,4.147655e+01,0.000000,-3.402264,0.290251,-0.000036,0.000000,-0.000011,0.000040
8,2.302719,-0.209885,-0.142592,1.390972,1.819285e+02,-0.909693,4.227592,-0.511594,3.990109e+01,0.000000,1.854519,-1.768852,-0.000087,0.000000,-0.000039,0.000002
9,-0.055176,-1.097004,-4.462851,-1.087149,1.869617e+02,0.395120,1.074678,0.667122,3.732614e+01,0.000000,-1.657733,1.984083,0.004780,0.000000,0.002562,-0.000257


In [64]:
bat1_2.errors_array.plot()

In [60]:
bat1_2.errors_array.plot()

In [50]:
table1_2=BatteryTable(bat1_2).data

,Stage 1,Stage 2,Stage 3
"Org in, ton/hr",173.200000,174.119022,187.935110
"y1 in, %",6.000000,5.931305,6.432133
"y2 in, %",6.000000,6.698796,12.540453
"y3 in, %",1.000000,1.027552,0.709202
"Aq in, ton/hr",35.238229,42.132050,50.000000
"x1 in, %",0.992031,0.992031,1.000000
"x2 in, %",52.020275,54.224967,60.000000
"x3 in, %",1.876323,0.810624,0.100000
"Org out, ton/hr",195.686030,183.279962,200.072291
"y1 out, %",8.580925,6.956736,7.620483


In [ ]:
# @markdown plots
fig = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

# visib='legendonly'
visib=True
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat1_2
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil6_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01), aspectratio=dict(x=1, y=1, z=1))
fig.update_scenes(camera_projection_type="orthographic")

fig.show()

### 7 - Washing2 - water_org=12

In [ ]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [ ]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [ ]:
mesh_predict_x=np.array([EQUIL_rotem_washing3_2([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3_2([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [ ]:
bat7_1=LLC.Battery(stages_num=6, Oin=90, Ain=29, yin=[10, 14, .07], xin=[0, 0.1, .02], EQUIL=EQUIL_rotem_washing3_2, eff=.7)
bat7_1.runs

In [ ]:
table7_1=LLC.BatteryTable(bat7_1).data

In [ ]:
fig_equil7_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat7_1
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil7_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil7_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil7_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil7_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil7_1.update_scenes(camera_projection_type="orthographic")

fig_equil7_1.show()

### 8 - Washing3 - compare with plant data

Profile data

In [ ]:
data=CopyPasteToArray('''
x	y
0	0.0531
3.01	0.26
10.8	1.22
20.69	6.02
33.12	7.77
34.9	8.86
35.81	10.3
''')
profile_data1=pd.DataFrame(data[1:], columns=data[0])

Plant data

In [ ]:
PS_P2O5=11.6
WS_P2O5=0.2
W_flow=23.4
Prod_flow=48.4 # =36.7*1.32
Prod_P2O5=37.6

PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
WS_flow=PS_flow+W_flow-Prod_flow
P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5)
print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

In [ ]:
summary_table8=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",6,'---', '---'],
                        index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                        columns=['Plant data'])
summary_table8

[0.76853034 3.57952069] root,
[0.75617284 3.26131687] direct,
[0.77263374 3.61111111] direct *10,
[0.77057613 3.48536808 2.34910837] direct entrain

In [ ]:
bat8_1=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=.77, convergence=1E-3, entrainment_perc_out=[3.6,0])
bat8_1.runs

In [ ]:
bat=bat8_1
summary_table8['Model data-0.75-3']=[bat.Oin,bat.yin[1]/98*71,bat.Oout,bat.yout[1]/98*71,bat.Ain,bat.Aout,bat.xout[1]/98*71,f"{bat.Aout*bat.xout[1]/(bat.Oin*bat.yin[1]):.1%}", f"{bat.stages_num:.0f}", bat.eff_list[0,1], str(bat.EQUIL)[17:-2]]
summary_table8

In [ ]:
table8_1=LLC.BatteryTable(bat8_1).data

In [ ]:
## P2O5
bat=bat8_1
summary_table=summary_table8
fig8_1 = go.Figure()
n=1 #H3PO4
fig8_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig8_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig8_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig8_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))
fig8_1.add_trace(go.Scatter(x=profile_data1.x, y=profile_data1.y, name='Plant profile data', marker_color='grey', visible='legendonly'))

fig8_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig8_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig8_1.update_layout(title='', width=1200, height=600,)
fig8_1.show()

In [ ]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [ ]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [ ]:
mesh_predict_x=np.array([EQUIL_rotem_washing3_3([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3_3([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [ ]:
fig_equil8_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat8_1
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil8_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil8_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil8_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil8_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil8_1.update_scenes(camera_projection_type="orthographic")

fig_equil8_1.show()

### 10 - Washing5 - optimize.direct()

Plant data

In [ ]:
PS_P2O5=11.6
WS_P2O5=0.2
W_flow=23.4
Prod_flow=48.4 # =36.7*1.32
Prod_P2O5=37.6

PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
WS_flow=PS_flow+W_flow-Prod_flow
P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5)
print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

In [ ]:
summary_table10=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",6,'---', '---'],
                        index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                        columns=['Plant data'])
summary_table10

#### slope(entr,eff)

נמצא שהתלות של השיפוע של קו הפעולה תלוי באחוז סחיפות בקשר ליניארי וביעילות בקשר
מרובע. מחשבים פרמטרים של פונקציה זאת

slope = a + b * entr + c * eff + d * eff^2

בשלב ראשון מוסיפים עמודה עם יעילות בריבוע

מתוך הריכוזים של התהליך מחשבים שיפוע של התהליך. מוצאים קשר בין סחיפות ויעילות.

entrain=(slope_proc-C[0]-C[2]*eff-C[3]*eff**2)/C[1]

האופטימיזציה הופכת לאחת עם משתנה בודד.

In [ ]:
## P2O5
summary_table=summary_table10
fig10_1 = go.Figure()
n=1 #H3PO4

for entr in [2.6, 3.6, 4.6]:
  for eff in [0.67,0.77,0.87]:
    bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
    print ('entr:', entr, '  eff:', eff, '  slope:', (bat.yin[1]-bat.yout[1])/(bat.xout[1]-bat.xin[1]))
    fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          # text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='entr:'+str(entr)+', '+'eff:'+str(eff)))

bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=.77, convergence=1E-3, entrainment_perc_out=[3.6,0])
fig10_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))

fig10_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig10_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig10_1.update_layout(title='', width=1200, height=600,)
fig10_1.show()

In [ ]:
slopes=[]
for entr in [3.6, 2.6, 4.6]: # לינארי -- מספיק 2
  for eff in [0.77,0.67,0.87]: # פורבולה -- צריך 3
    bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
    slope=(bat.yin[1]-bat.yout[1])/(bat.xout[1]-bat.xin[1]) # שיפוע של קו הפעולה
    slopes.append([entr, eff, slope])
    # print ('entr:', entr, '  eff:', eff, '  slope:', slope)
arr=np.array(slopes)
arr

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=arr[:,0], y=arr[:,1], z=arr[:,2], mode = "markers"))
fig.show()

In [ ]:
arr_mod=np.c_[np.ones([len(arr),1]),arr[:,0:2], arr[:,1]**2, arr[:,2]]
arr_mod

In [ ]:
rows=[0,1,2,3]
A=arr_mod[rows,0:4]
B=arr_mod[rows,4]
C=np.dot(np.linalg.inv(A),B) # מחשבים ערכים של עמודה אחרונה (קו הפעולה) מתוך עמודות ראשונות
C

In [ ]:
pred=np.dot(arr_mod[:,0:4],C)
pred

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=arr[:,0], y=arr[:,1], z=arr[:,2], mode = "markers"))
# fig.add_trace(go.Scatter3d(x=arr[rows,0], y=arr[rows,1], z=pred[rows], mode = "markers"))
fig.add_trace(go.Scatter3d(x=arr[:,0], y=arr[:,1], z=pred, mode = "markers"))

fig.show()

#### together

Plant data

In [ ]:
PS_P2O5=11.6
WS_P2O5=0.2
W_flow=23.4
Prod_flow=48.4 # =36.7*1.32
Prod_P2O5=37.6
PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
WS_flow=PS_flow+W_flow-Prod_flow
P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5)
print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

slope_proc=(PS_P2O5-WS_P2O5)/Prod_P2O5

In [ ]:
summary_table10_1=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",6,'---', '---'],
                        index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                        columns=['Plant data'])
summary_table10_1

Slope(entrain,eff)

נמצא שהתלות של השיפוע של קו הפעולה תלוי באחוז סחיפות בקשר ליניארי וביעילות בקשר
מרובע. מחשבים פרמטרים של פונקציה זאת

slope = a + b * entr + c * eff + d * eff^2

בשלב ראשון מוסיפים עמודה עם יעילות בריבוע

מתוך הריכוזים של התהליך מחשבים שיפוע של התהליך. מוצאים קשר בין סחיפות ויעילות.

entrain=(slope_proc-C[0]-C[2]*eff-C[3]*eff**2)/C[1]

האופטימיזציה הופכת לאחת עם משתנה בודד.

In [ ]:
slopes=[]
entr_initial, eff_init = 3.6, 0.77
for entr in [entr_initial, entr_initial*.75]: # לינארי -- מספיק 2
  for eff in [eff_init,eff_init*.85,eff_init*1.1]: # פורבולה -- צריך 3
    bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
    slope=(bat.yin[1]-bat.yout[1])/(bat.xout[1]-bat.xin[1])
    slopes.append([entr, eff, slope])
    # print ('entr:', entr, '  eff:', eff, '  slope:', slope)
arr=np.array(slopes)
arr_mod=np.c_[np.ones([len(arr),1]),arr[:,0:2], arr[:,1]**2, arr[:,2]] # entrainment approximated to porabola

In [ ]:
rows=[0,1,2,3] # 4 rows for 4 unknowns
A=arr_mod[rows,0:4]
B=arr_mod[rows,4]
C=np.dot(np.linalg.inv(A),B) # פתרון מערכת משוואות.  מחשבים ערכים של עמודה אחרונה (קו הפעולה) מתוך עמודות ראשונות

optimize

optimization with 1 parameter

במקום להריץ אופטימיזציה עם 2 משתנים: יעילות וסחיפות, זה רץ על פרמטר בודד -- יעילות. הסחיפות מחושבות מתוך תלות בין סחיפות ושיפוע קו פעולה. תלות זאת חושבה לפני זה.

In [ ]:
def P2O5_calc3_1(x):
  eff = x[0]
  entrain=(slope_proc-C[0]-C[2]*eff-C[3]*eff**2)/C[1]
  bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff, convergence=1E-3, entrainment_perc_out=[entrain,0])
  # error1=abs((WS_P2O5-bat.yout[1]/98*71)/WS_P2O5)
  error=abs((Prod_P2O5-bat.xout[1]/98*71)/Prod_P2O5)
  print (eff, entrain, bat.yout[1]/98*71, bat.xout[1]/98*71, error, bat.runs, ',')

  return error

In [ ]:
bounds=[(0.5,1)]
sol10_1=optimize.direct(P2O5_calc3_1, bounds=bounds, f_min=0, f_min_rtol=0.001, maxiter=100, locally_biased=False)
print (sol10_1.x, sol10_1.fun)

In [ ]:
eff10_1=sol10_1.x[0]
entrain10_1=entrain=(slope_proc-C[0]-C[2]*eff10_1-C[3]*eff10_1**2)/C[1]

optimal calcul

In [ ]:
bat10_1=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff10_1, convergence=1E-3, entrainment_perc_out=[entrain10_1,0])
bat10_1.runs

In [ ]:
bat=bat10_1
summary_table10_1['Model data']=[bat.Oin,bat.yin[1]/98*71,bat.Oout,bat.yout[1]/98*71,bat.Ain,bat.Aout,bat.xout[1]/98*71,f"{bat.Aout*bat.xout[1]/(bat.Oin*bat.yin[1]):.1%}", f"{bat.stages_num:.0f}", bat.eff_list[0,1], str(bat.EQUIL)[17:-2]]
summary_table10_1

In [ ]:
table10_1=LLC.BatteryTable(bat10_1).data

#### vizual

In [ ]:
## P2O5
bat=bat10_1
summary_table=summary_table10_1
fig10_1 = go.Figure()
n=1 #H3PO4
fig10_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))
fig10_1.add_trace(go.Scatter(x=profile_data1.x, y=profile_data1.y, name='Plant profile data', marker_color='grey', visible='legendonly'))

fig10_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig10_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig10_1.update_layout(title='', width=1200, height=600,)
fig10_1.show()

In [ ]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [ ]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [ ]:
mesh_predict_x=np.array([EQUIL_rotem_washing3_3([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3_3([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [ ]:
fig_equil10_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat10_1
# H3PO4_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil10_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil10_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil10_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil10_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil10_1.update_scenes(camera_projection_type="orthographic")

fig_equil10_1.show()

# Profile

In [ ]:
data22=CopyPasteToArray('''
H3PO4_aq	SO4_aq	SO4_org	Unit
4.15	0.147	0.0626	571
14.91	0.0806	0.0852	573
28.56	0.0329	0.0787	575
45.71	0.0106	0.0664	579
48.17	0.0078	0.0631	581
49.43	0.0058	0.0641	5150
''')

data22=pd.DataFrame(data22[1:], columns=data22[0])

In [ ]:
data22

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='Tests'))
fig.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0],
                                     mode='markers',marker_color='gray', marker_size=1, name='Equilibrium surface', visible=True))
fig.add_trace(go.Scatter3d(x=data22.H3PO4_aq, y=data22.SO4_aq, z=data22.SO4_org,
                                     mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)],
                                     textposition ='bottom center', name='Profile', marker_color='green'))

fig.update_layout(title='', autosize=True,
                  # width=980,
                  height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq',  zaxis_title='SO4_org', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig.update_scenes(camera_projection_type="orthographic")

# fig.write_html("Profile_SO4.html")
fig.show()